### moving from jinja to python.py prompts 
i though i will use jinja for prompts here but forget that  jinja is not made for routing , my prompts 
here looks less like template and more like dynamic routing prompts 

 i was using lots and lots of if else condition 
say for 20 agent , checking 20 if else condition make system slow so i am moving 


In [2]:

"""    
    Returns the prompt text for a given role and agent type.
    Fills in any required variables provided as kwargs.
    
    Parameters:
        role (str): 'system' or 'user'
        agent_type (str): e.g., 'validator_agent', 'researcher_agent'
        **kwargs: required variables for the prompt (if any)
        
    Returns:
        str: Fully rendered prompt text
    

"""

from prompt import get_prompt

## create an MCP 

In [3]:
from typing import Union
from pydantic import BaseModel , field_validator , Field 
from typing import Dict, Any , Union , TypedDict
import uuid


class ValidatorContext(BaseModel):
    task: str
    source_summary: str
    draft_post: str



class MCPMessage(BaseModel):
    protocol_version: str = "1.0"
    sender: str= Field(min_length=3)
    content: Any
    metadata: Dict[str, Any] = Field(default_factory=dict)


def create_mcp_message(sender: str ,  content: Any, metadata=None) -> MCPMessage:
    """Create a message in MCP format."""

    
    if not isinstance(content, (str, dict, ValidatorContext)):
        raise TypeError(
            f"content must be str, dict, or ValidatorContext; got {type(content).__name__}"
        )

        

    metadata = metadata or {} 
    metadata.setdefault("task_id", str(uuid.uuid4()))
    metadata.setdefault("parents",[])

    return MCPMessage(
        sender=sender,
        content=content,
        metadata=metadata   
    )

mcp_answer = create_mcp_message("agent_1", "Hello, this is a test message.", {"priority": "high" , "parents": ["writer-agent"]})
print(mcp_answer.sender)
print(mcp_answer.content)
print(mcp_answer.metadata)

agent_1
Hello, this is a test message.
{'priority': 'high', 'parents': ['writer-agent'], 'task_id': '70658341-83ef-4a4d-8fab-066c27ddbc5c'}


####  not using any fallback logics or anything or simplicity 
#### i found i  will only use those if topic specific or  in end to end projects

In [4]:
def agent_context_librarian(mcp_message):
    """
        Retrieves the appropriate Semantic Blueprint from the Context Library.
    """

    # ----------- type check --------------------
    if not isinstance(mcp_message, dict):
        raise TypeError("mcp_message must be a dictionary")

    # --- structure check ---
    if "content" not in mcp_message or not isinstance(mcp_message["content"], dict):
        raise KeyError("mcp_message must contain a 'content' dictionary")

    if "intent_query" not in mcp_message["content"]:
        raise KeyError("mcp_message['content'] must contain 'intent_query'")

    if not isinstance(mcp_message["content"]["intent_query"], str):
        raise TypeError("'intent_query' must be a string")

        
    print("\\n[Librarian] Activated. Analyzing intent...")
    requested_intent = mcp_message['content']['intent_query']
    results = query_pinecone(requested_intent, NAMESPACE_CONTEXT, top_k=1)
    if results:
        match = results[0]
        print(f"[Librarian] Found blueprint '{match['id']}' (Score:{match['score']:.2f})")
        blueprint_json = match['metadata']['blueprint_json']
        content = {"blueprint": blueprint_json}

    else:
        print("[Librarian] No specific blueprint found. Returning default.")
        content = {"blueprint": json.dumps({"instruction": "Generate the content neutrally."})}
    return create_mcp_message("Librarian", content)



## research agent 


In [8]:
INDEX_NAME = 'genai-mas-mcp-ch3'
NAMESPACE_KNOWLEDGE = "KnowledgeStore"
NAMESPACE_CONTEXT = "ContextLibrary"

In [ ]:
## pine cone query 
from helper import query_pinecone 
## takes  query_text , namespaces and tok k (optional, because defult is 1)
## and return index query ... response['matches']

query_pinecone(topic = "writer agent")

results = query_pinecone(topic, NAMESPACE_KNOWLEDGE, top_k=3)

In [ ]:
def get_or_create_index(
    pc,
    index_name: str,
    embedding_dim: int,
    namespaces_to_clear: List[str] | None = None,
    metric: str = "cosine",
    cloud: str = "aws",
    region: str = "us-east-1",
    delete_timeout_seconds: int = 120,
    poll_interval: int = 2,
):

Index 'genai-mas-mcp-ch3' already exists.


In [9]:
from helper import get_or_create_index

ImportError: cannot import name 'get_or_create_index' from 'helper' (/home/ujjwal/projects/Context-Enginnering/helper.py)

In [ ]:
from prompt import get_prompt
get_prompt(role= "user" , 
           agent_type="research_synthesis_ai",
           topic = "this this" ,

           sources="thsihs" )

'Topic: this this\n\nSources:\nthsihs'

In [ ]:
results = query_pinecone(topic, NAMESPACE_KNOWLEDGE, top_k=3)


source_texts = [match['metadata']['text'] for match in results]


In [6]:
results = query_pinecone(topic, NAMESPACE_KNOWLEDGE, top_k=3)

NameError: name 'query_pinecone' is not defined

In [5]:
from prompt import get_prompt
get_prompt(role= "system" , 
           agent_type="research_synthesis_ai",
           topic = "this this" ,
           sources="thsihs" )

"System Prompt (Research Synthesis AI):\nYou are an expert research synthesis AI. Synthesize the provided source texts into a concise, bullet-pointed summary relevant to the user's topic. Focus strictly on the facts provided in the sources. Do not add outside information."

In [ ]:
def agent_researcher(mcp_message):

    if not isinstance(mcp_message, dict):
        raise TypeError("mcp_message must be a dictionary")

    # --- structure check ---
    if "content" not in mcp_message or not isinstance(mcp_message["content"], dict):
        raise KeyError("mcp_message must contain a 'content' dictionary")

    if "intent_query" not in mcp_message["content"]:
        raise KeyError("mcp_message['content'] must contain 'intent_query'")

    if not isinstance(mcp_message["content"]["topic_query"], str):
        raise TypeError("'topic_query' must be a string")

    
    print("Researcher Activated: Investigating topic ...")
    topic=  mcp_message['content']['topic_query']
    source_texts= get_prompt(           role= "system" , 
                                         agent_type="research_synthesis_ai", ) 
    
    
    
    


    